In [58]:
import torch
import pandas as pd
import os
from tqdm import tqdm

In [59]:
torch.cuda.device_count()

1

In [60]:
#Import Bio_ClinicalBERT tokenizer
from transformers import *
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/wud2/.cache/huggingface/hub/models--emilyalsentzer--Bio_ClinicalBERT/snapshots/d5892b39a4adaed74b92212a44081509db72f87b/config.json
Model config BertConfig {
  "_name_or_path": "emilyalsentzer/Bio_ClinicalBERT",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /home/wud2/.cache/huggingface/hub/models--emilyalsentzer--Bio_ClinicalBERT/snapshots/d5

In [61]:
print(tokenizer)

BertTokenizerFast(name_or_path='emilyalsentzer/Bio_ClinicalBERT', vocab_size=28996, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [62]:
#Import Phenotagger labelled output notes
data_path = '/mnt/isilon/wang_lab/jingye/projects/Phenotagger/test_output/'
files = [f for f in os.listdir(data_path) if not f.startswith('.')]
files.remove('dirty_notes')
files,len(files)

(['3066421',
  '2872026',
  '2811502',
  '2774322',
  '2994298',
  '2072421',
  '1900984',
  '3007901',
  '2337814',
  '566069',
  '3078202',
  '3009680',
  '2115454',
  '2281712',
  '2885872',
  '2879288',
  '3215647',
  '3270176',
  '3162228',
  '2212239',
  '2678658',
  '2690635',
  '1825429',
  '768495',
  '1990062',
  '2304032',
  '3095519',
  '2189612',
  '3074835',
  '1744447',
  '2497106',
  '3003726',
  '3331757',
  '2698061',
  '2583990',
  '1718646',
  '3101668',
  '3029407',
  '2629367',
  '3016952',
  '2612425',
  '2746127',
  '1642177',
  '2553354',
  '358441',
  '3095296',
  '2885266',
  '2218672',
  '3112974',
  '3102250',
  '2411493',
  '2822150',
  '2695474',
  '3007773',
  '2296161',
  '1928141',
  '2634712',
  '2712677',
  '2931085',
  '3015472',
  '3066284',
  '2818906',
  '3255976',
  '2486727',
  '3050571',
  '3162532',
  '2395487',
  '2000048',
  '3153820',
  '2502006',
  '2644173',
  '3223977',
  '3092476',
  '2886400',
  '2873497',
  '2038538',
  '2541913',
  

In [11]:
def prepare_dataset(files):
    notes = []
    pheno_list = []
    
    for f in tqdm(files):
        file = open('/mnt/isilon/wang_lab/jingye/projects/Phenotagger/test_output/'+f, 'r')
        text = file.readlines()
        note = text[0][text[0].find('|t|')+3:]
        notes.append(note)
        pheno = []
        for line in text[3:]:
            if line != '\n':
                line = line[:-1].split('\t')
                pheno.append(line[-2:])
            else:
                break
        pheno_list.append(pheno)
    return notes, pheno_list

In [12]:
train_files = files[:-100]
test_files = files[-100:]
notes, pheno_list = prepare_dataset(train_files)
test_notes, test_pheno_list = prepare_dataset(test_files)

100%|██████████| 100/100 [00:00<00:00, 988.47it/s]


In [13]:
#Sanity Check
note = notes[1024]
pheno = pheno_list[1024]
print(note+'\n')
for i, j in pheno:
    print(i , j)
file = open('/mnt/isilon/wang_lab/jingye/projects/Phenotagger/test_output/'+files[1024], 'r')
print(file.readlines())

Grant is a 10 year old male with developmental delay, autism, camptodactyly, intellectual disability, dysmorphic features, overweight. Previous testing has included normal PWS methylation, normal Fragile X, 9.4Mb de novo deletion of chromosome 1p21.1p21.3 identified on arrayPrior to today's visit, the family did not view the educational module on genetics and genetic testing. During today's consult, exome sequencing (ES) and several of its benefits and limitations were discussed with Grant's parents. Exome sequencing is the most comprehensive genetic test that is clinically available and has the highest chance of identifying an underlying diagnosis in Grant.  It is utilized as a diagnostic tool when trying to identify a genetic cause for an individual's symptoms or disease when one's findings do not fit a specific diagnosis. This is because more than 85% of genetic changes that cause inherited conditions are found in the exome, even though the exome constitutes just about ~2% of the wh

In [15]:
#Sample tokenizer split of a whole sentence
ans = tokenizer('he has a longstanding history of rheumatoid arthritis that involved small and large joints in her body')
tokenizer.convert_ids_to_tokens(ans.input_ids)

['[CLS]',
 'he',
 'has',
 'a',
 'long',
 '##standing',
 'history',
 'of',
 'r',
 '##he',
 '##uma',
 '##to',
 '##id',
 'art',
 '##hr',
 '##itis',
 'that',
 'involved',
 'small',
 'and',
 'large',
 'joints',
 'in',
 'her',
 'body',
 '[SEP]']

In [16]:
#Tokenize notes
note_tokenized = tokenizer(notes, padding='max_length', max_length=512, truncation=True, return_tensors="pt")

In [17]:
#{key : torch.tensor(val[0]) for key, val in note_tokenized.items()}

/tmp/ipykernel_2578480/4236103152.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  {key : torch.tensor(val[0]) for key, val in note_tokenized.items()}


{'input_ids': tensor([  101,  5837,  5086,  1110,   170,   127,  1214,  1385,  2581,  1114,
           181,   118, 11641,   172, 17868,   113,   188,   120,   185,  6949,
           114,  1105,  3694, 26600,   177,  1183, 18299, 22992,  1112,  1218,
          1112,   171,  1233,  5354,  6451,  1766,  4313, 10396,  1306,   113,
           188,   120,   185,  1137,  4313, 15622, 16844,   114,   119,  1229,
          5837,  5086,  1674,  1136,  1138,  1168,  8649,  5677,  1449,  5408,
           117,  1114,  1159,  1122,  1144,  1561,  6281,  1115,  1119,  1144,
          4265, 16700, 15308,   119,  5837,  5086,  5315,   178,  8043,  1105,
          7623,  3776,  3750,   119,  1119,  1145,  1144, 15360,  4035, 22962,
           177,  1183, 18299, 22992,  1105,   182,  1361, 10182,  8867, 13622,
         21470,  9505,   113,   185, 10294,  4814,  4252,  2599, 26465,  1818,
           117,  4091,   177, 24312,  3702,  5474,   117,   177,  1183, 11439,
         11357,   117, 10558,  5968, 14

In [18]:
idx_s, idx_e = note_tokenized.char_to_token(1024,33), note_tokenized.char_to_token(1024,51)
tokenizer.convert_ids_to_tokens(note_tokenized["input_ids"][1024])[idx_s: idx_e+1]

['developmental', 'delay']

In [19]:
tokenizer.convert_ids_to_tokens(note_tokenized["input_ids"][1024])

['[CLS]',
 'grant',
 'is',
 'a',
 '10',
 'year',
 'old',
 'male',
 'with',
 'developmental',
 'delay',
 ',',
 'autism',
 ',',
 'camp',
 '##to',
 '##da',
 '##ct',
 '##yl',
 '##y',
 ',',
 'intellectual',
 'disability',
 ',',
 'd',
 '##ys',
 '##morphic',
 'features',
 ',',
 'over',
 '##weight',
 '.',
 'previous',
 'testing',
 'has',
 'included',
 'normal',
 'p',
 '##ws',
 'met',
 '##hyl',
 '##ation',
 ',',
 'normal',
 'fragile',
 'x',
 ',',
 '9',
 '.',
 '4',
 '##mb',
 'de',
 'no',
 '##vo',
 'del',
 '##eti',
 '##on',
 'of',
 'chromosome',
 '1',
 '##p',
 '##21',
 '.',
 '1',
 '##p',
 '##21',
 '.',
 '3',
 'identified',
 'on',
 'array',
 '##p',
 '##rio',
 '##r',
 'to',
 'today',
 "'",
 's',
 'visit',
 ',',
 'the',
 'family',
 'did',
 'not',
 'view',
 'the',
 'educational',
 'module',
 'on',
 'genetics',
 'and',
 'genetic',
 'testing',
 '.',
 'during',
 'today',
 "'",
 's',
 'consult',
 ',',
 'ex',
 '##ome',
 'se',
 '##quencing',
 '(',
 'es',
 ')',
 'and',
 'several',
 'of',
 'its',
 'benefits'

In [22]:
#Function to label phenotypic terms 
#Specifically, all the phenotypic term words (e.g. both "developmental" and "delay") will be labelled as 1. 
def align_label(tokenized_input, pheno_list):

        # word_ids = tokenized_input.word_ids()
    labels = []
    for idx_in_batch in range(len(pheno_list)):

        label_ids = [-100]*512
        try:
            pad_id = tokenized_input["input_ids"][idx_in_batch].tolist().index(0)
        except ValueError:
            pad_id = 512
        
        label_ids[1:pad_id-1] = [0 for _ in label_ids[1:pad_id-1]]

        pheno = pheno_list[idx_in_batch]
        for start, end in pheno:
            idx_s, idx_e = tokenized_input.char_to_token(idx_in_batch, int(start)), tokenized_input.char_to_token(idx_in_batch, int(end)-1)
            if idx_s == None:
                break
            if idx_e == None:
                label_ids[idx_s:pad_id-1] = [1 for _ in label_ids[idx_s:pad_id-1]]
                break
            for i in range(idx_s, idx_e+1):
                 label_ids[i] = 1

        labels.append(label_ids)

    return labels

In [24]:
#note_tokenized

{'input_ids': tensor([[  101,  5837,  5086,  ...,   117,  1343,   102],
        [  101,  1177, 27008,  ...,   188,   117,   102],
        [  101,  7586,  2883,  ...,  1718,   119,   102],
        ...,
        [  101,   188,  2707,  ..., 17895,  1348,   102],
        [  101,  1189,  2568,  ...,  2492,  1115,   102],
        [  101,   179,  6397,  ...,  2656,  1104,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [25]:
#pheno_list

[[['70', '90'],
  ['105', '119'],
  ['251', '278'],
  ['348', '372'],
  ['403', '419'],
  ['421', '440'],
  ['442', '451'],
  ['453', '481'],
  ['483', '493'],
  ['726', '742'],
  ['744', '755'],
  ['744', '760'],
  ['750', '760'],
  ['762', '774'],
  ['776', '787'],
  ['3083', '3103'],
  ['3125', '3134'],
  ['3156', '3182'],
  ['3671', '3677'],
  ['3700', '3722'],
  ['3730', '3744'],
  ['3754', '3765'],
  ['3798', '3818'],
  ['3821', '3843'],
  ['3831', '3843'],
  ['3845', '3861'],
  ['5769', '5788']],
 [['2440', '2460'],
  ['2482', '2491'],
  ['2513', '2539'],
  ['3021', '3027'],
  ['3050', '3065'],
  ['3073', '3087'],
  ['3097', '3108'],
  ['3142', '3162'],
  ['3170', '3192'],
  ['3180', '3192'],
  ['3194', '3210'],
  ['3277', '3293'],
  ['5580', '5599']],
 [['46', '61'],
  ['63', '82'],
  ['87', '93'],
  ['87', '111'],
  ['143', '149'],
  ['200', '206'],
  ['285', '291'],
  ['485', '491'],
  ['582', '588'],
  ['645', '651'],
  ['645', '670'],
  ['768', '774'],
  ['768', '792'],
  [

In [26]:
label_id = align_label(note_tokenized, pheno_list)

In [27]:
len(label_id)

2843

In [28]:
note_tokenized.char_to_token(1024,123), note_tokenized.char_to_token(1024,132)

(29, 30)

In [41]:
label_id[1024][28:32]

[0, 1, 1, 0]

In [31]:
tokenizer.convert_ids_to_tokens(note_tokenized["input_ids"][1024])[29:31]

['over', '##weight']

In [32]:
class DataSequence(torch.utils.data.Dataset):

    def __init__(self, notes ,pheno_list):

        self.tokenized_notes = tokenizer(notes,
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
        self.labels = align_label(self.tokenized_notes, pheno_list)

    def __len__(self):

        return len(self.labels)

    # def get_batch_data(self, idx):

    #     return self.tokenized_notes[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):
        batch_data = {key : torch.tensor(val[idx]) for key, val in self.tokenized_notes.items()}
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [34]:
train_dataset = DataSequence(notes, pheno_list)
val_dataset = DataSequence(test_notes, test_pheno_list)

In [35]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, num_workers=1 #usually = No. of cpu
                                               , batch_size=16, shuffle=True)
len(train_dataloader)

178

In [37]:
from transformers import BertForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels = 2)

model

loading configuration file config.json from cache at /home/wud2/.cache/huggingface/hub/models--emilyalsentzer--Bio_ClinicalBERT/snapshots/d5892b39a4adaed74b92212a44081509db72f87b/config.json
Model config BertConfig {
  "_name_or_path": "emilyalsentzer/Bio_ClinicalBERT",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /home/wud2/.cache/huggingface/hub/models--emilyalsentzer--Bio_ClinicalBERT/snapshots/d5892b39a4adaed74b92212a44081509db72f87b/pytorch_model.bin
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from th

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [38]:
from torch.utils.data import DataLoader
from torch.optim import SGD, AdamW

In [39]:
tokenizer.save_pretrained('./tokenizer')
model = torch.nn.DataParallel(model)

tokenizer config file saved in ./tokenizer/tokenizer_config.json
Special tokens file saved in ./tokenizer/special_tokens_map.json


In [40]:
def train_loop(model, notes, pheno_list, test_notes = None, test_pheno_list = None):

    if test_notes == None:
        test_notes = notes
        test_pheno_list = pheno_list
    # train_dataset = DataSequence(df_train)
    # val_dataset = DataSequence(df_val)
    dataset = DataSequence(notes, pheno_list)            
    test_dataset = DataSequence(test_notes, test_pheno_list)

    # train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    # val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)
    dataloader = DataLoader(dataset, num_workers=1, batch_size=BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(test_dataset, num_workers=1, batch_size=BATCH_SIZE, shuffle=True)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = SGD(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):
        nan_train = 0
        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for count, [train_data, train_label] in enumerate(tqdm(dataloader)):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].to(device)
            input_id = train_data['input_ids'].to(device)

            optimizer.zero_grad()
#             output = model(input_ids=input_id, attention_mask=mask, labels=train_label)
#             print(output.loss)
            loss, logits = model(input_ids=input_id, attention_mask=mask, labels=train_label, return_dict=False)
#             print(loss.shape,logits.shape)
            for i in range(logits.shape[0]):
                logits_clean = logits[i][train_label[i] != -100]
                label_clean = train_label[i][train_label[i] != -100]
                
                if len(label_clean) == 0:
                    nan_train += 1
                    continue
                
                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                
                total_acc_train += acc
                total_loss_train += loss.sum().item()

            loss.sum().backward()
            optimizer.step()
            
            if count % 5 == 0 and count > 0:
                print(loss.sum()/torch.cuda.device_count())

        model.eval()

        nan_test = 0
        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in test_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].to(device)
            input_id = val_data['input_ids'].to(device)

            loss, logits = model(input_ids=input_id, attention_mask=mask, labels=val_label, return_dict=False)

            for i in range(logits.shape[0]):

                logits_clean = logits[i][val_label[i] != -100]
                label_clean = val_label[i][val_label[i] != -100]
                if len(label_clean) == 0:
                    nan_test += 1
                    continue

                predictions = logits_clean.argmax(dim=1)
                acc = (predictions == label_clean).float().mean()
                total_acc_val += acc
                total_loss_val += loss.sum().item()

#         val_accuracy = total_acc_val / len(df_val)
#         val_loss = total_loss_val / len(df_val)

        print(
            f'length : {len(notes)} | Epochs: {epoch_num + 1} | Loss: {total_loss_train / (len(notes)-nan_train): .3f} | Accuracy: {total_acc_train / (len(notes)-nan_train): .3f} |' + 
            f'Val_Loss: {total_loss_val / (len(test_notes) - nan_test): .3f} | Accuracy: {total_acc_val / (len(test_notes) - nan_test): .3f}')

LEARNING_RATE = 5e-4
EPOCHS = 30
BATCH_SIZE = 64

train_loop(model, notes, pheno_list, test_notes, test_pheno_list)

  0%|          | 0/45 [00:00<?, ?it/s]/tmp/ipykernel_2578480/1618731302.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_data = {key : torch.tensor(val[idx]) for key, val in self.tokenized_notes.items()}
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 13%|█▎        | 6/45 [00:07<00:57,  1.46s/it]

tensor(0.5127, device='cuda:0', grad_fn=<DivBackward0>)


 24%|██▍       | 11/45 [00:13<00:47,  1.40s/it]

tensor(0.4169, device='cuda:0', grad_fn=<DivBackward0>)


 36%|███▌      | 16/45 [00:19<00:40,  1.39s/it]

...

tensor(0.1279, device='cuda:0', grad_fn=<DivBackward0>)


 91%|█████████ | 41/45 [00:49<00:05,  1.41s/it]

tensor(0.1450, device='cuda:0', grad_fn=<DivBackward0>)


100%|██████████| 45/45 [00:52<00:00,  1.17s/it]
/tmp/ipykernel_2578480/1618731302.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_data = {key : torch.tensor(val[idx]) for key, val in self.tokenized_notes.items()}
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


length : 2843 | Epochs: 30 | Loss:  0.133 | Accuracy:  0.941 |Val_Loss:  0.132 | Accuracy:  0.949


In [64]:
#Save the model to your own path
model.module.save_pretrained('./model/epoch30')

In [47]:
#one sentence only
def align_ids(tokenized_input):

        # word_ids = tokenized_input.word_ids()
    input_ids = tokenized_input["input_ids"]

    label_ids = [-100]*512
    try:
        pad_id = input_ids[0].tolist().index(0)
    except ValueError:
        pad_id = 512
    
    label_ids[1:pad_id-1] = [0 for _ in label_ids[1:pad_id-1]]


    return label_ids

In [48]:
note_tokenized

{'input_ids': tensor([[  101,  5837,  5086,  ...,   117,  1343,   102],
        [  101,  1177, 27008,  ...,   188,   117,   102],
        [  101,  7586,  2883,  ...,  1718,   119,   102],
        ...,
        [  101,   188,  2707,  ..., 17895,  1348,   102],
        [  101,  1189,  2568,  ...,  2492,  1115,   102],
        [  101,   179,  6397,  ...,  2656,  1104,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [66]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
mask = note_tokenized['attention_mask'].to(device)
input_id = note_tokenized['input_ids'].to(device)
output = model(input_ids=input_id, attention_mask=mask)
logits = output.logits

valid_ids = torch.Tensor(align_ids(note_tokenized)).unsqueeze(0).to(device)
print(valid_ids.shape)
output = model(input_ids=input_id, attention_mask=mask)
logits = output.logits
sample = logits[0].unsqueeze(0)
print(sample.shape)
logits_clean = sample[valid_ids != -100]
logits_clean.shape

In [51]:
def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.tensor(align_ids(text)).to(device)

    output = model(input_ids=input_id, attention_mask=mask)
    logits = output.logits.squeeze(0)

    previous_word = None
    phenotype = []
    for i in range(len(logits)):
        if label_ids[i] == -100:
            continue
        if logits[i].argmax().item() == 0:
            continue
        
        current_word = text.token_to_word(i)
        if current_word == previous_word:
            continue
        pheno = sentence[text.word_to_chars(current_word).start : text.word_to_chars(current_word).end]

        if previous_word == None or current_word != (previous_word + 1):
#             print(text.word_to_chars(current_word).start)
            phenotype.append(pheno)
        else:
            phenotype[-1] = phenotype[-1] + ' ' + pheno
            

        previous_word = current_word

    # print(text.word_to_chars(text.token_to_word(510)))
    print(phenotype)
            


In [52]:
evaluate_one_text(model, test_notes[60])

['epistaxis', 'telangiectasia', 'telangiectasias', 'melanocytic hyperpigmented nevus', 'hyperhydrosis', 'epistaxis', 'HTN', 'epistaxis', 'malformation', 'venus malformation']


In [54]:
file = open('/mnt/isilon/wang_lab/jingye/projects/Phenotagger/test_output/'+ files[2903], 'r')
# print(file.readlines())

In [55]:
file.readlines()

["1795493|t|Ryleigh is a 18 year old female with long-standing hx of epistaxis and L palmar telangiectasia (now w/ 2 additional telangiectasias: above L lip and L eyebrow) who also has a melanocytic hyperpigmented nevus and focal hyperhydrosis of hands and feet leading to intermittent skin peeling.  She does not have a hx of GI or other unusual bleeding (aside from epistaxis), no known AVMs (but has not had any targeted MRA imaging), and no pulmonary HTN history.  There is no hx of ataxia, to suggest AT; on the contrary, Ryleigh is an accomplished athlete. The family hx is not highly indicative of HHT but there can be notable phenotypic variability and even decreased penetrance: her father has occasional epistaxis; her mother has two hemangiomas/small vascular lesions that have not required evaluation.Given Ryleigh's clinical findings, we certainly are in agreement that testing of HHT genes is clinically indicated for her. Given the overlap between HHT and CM-AVM (capillary malformatio

In [56]:
#Example testing sentence
sent = "Ryleigh is a 18 year old female with long-standing hx of epistaxis and L pal " + \
"telrngiectasia (now w/ 2 additional telangiectasias: above L lip and L eyebrow) who also has a melanocytic hyperpigmented nevus and focal hyperhydrosis of hands and feet leading to intermittent skin peeling.  She does not have a hx of GI or other unusual bleeding (aside from epistaxis), no known AVMs (but has not had any targeted MRA imaging), and no pulmonary HTN history.  There is no hx of ataxia, to suggest AT; on the contrary, Ryleigh is an accomplished athlete. The family hx is not highly indicative of HHT but there can be notable phenotypic variability and even decreased penetrance: her father has occasional epistaxis; her mother has two hemangiomas/small vascular lesions that have not required evaluation.Given Ryleigh's clinical findings, we certainly are in agreement that testing of HHT genes is clinically indicated for her. Given the overlap between HHT and CM-AVM (capillary malformation - arterial venus malformation) spectrum, we are recommending a full HHT gene panel that includes genes causing both phenotypes: ACVRL1, ENG, EPHB4, GDF2, RASA1, and SMAD4.HHT gene mutations are autosomal dominant; thus if Ryleigh's result is positive or inconclusive, testing of both parents will be recommended. If HHT testing were negative, that would not definitively r/o HHT diagnosis as the sensitivity of the 6-gene panel is ~85%."

In [57]:
#Investigate the evaluation example
evaluate_one_text(model, sent)

['epistaxis', 'telrngiectasia', 'telangiectasias', 'eyebrow', 'melanocytic hyperpigmented nevus', 'hyperhydrosis', 'intermittent', 'epistaxis', 'HTN', 'epistaxis', 'malformation', 'venus malformation']
